In [1]:
import enum
import threading
import uuid
import datetime
import random
import time

In [2]:
class channel(enum.Enum):
    inbound = 1
    email = 2

In [3]:
class event(enum.Enum):
    start = 1
    abandon = 2
    handle = 3
    end = 4

In [4]:
class queue:    
    def __init__(self, identification, name, channel_id, service_level_target, service_lever_duration):
        self.identification = identification
        self.name = name
        self.channel_id = channel_id
        self.service_level_target = service_level_target
        self.service_lever_duration = service_lever_duration
        self.contacts = []
        
    def put_contact(self, contact):
        self.contacts.append(contact)
        print({'contact_identification': contact['contact_identification'], 'queue': self.name, 'event_identification': event.start.name, 'event_created_at': datetime.datetime.utcnow()})
        
    def get_contact(self):
        return self.contacts.pop(0)
    
    def remove_contact(self, contact):
        self.contacts.remove(contact)

In [5]:
class agent:    
    def __init__(self, identification, name):
        self.identification = identification
        self.name = name

In [6]:
class agents_connected:    
    def __init__(self):
        self.number_of_agents_connected = []
    
    def add_agent(self, agent_identification):
        self.number_of_agents_connected.append(agent_identification)
        
    def remove_agent(self, agent_identification):
        self.number_of_agents_connected.remove(agent_identification)

In [7]:
def system_monitoring(queue):
    while True:
        if agents_connected.number_of_agents_connected:
            if queue.contacts:
                for contact in queue.contacts:
                    if  datetime.datetime.utcnow() >= contact['created_at'] + datetime.timedelta(seconds=contact['wait_time']):
                        try:
                            queue.remove_contact(contact)
                            print({'contact_identification': contact['contact_identification'], 'queue': queue.name, 'event_identification': event.abandon.name, 'event_created_at': datetime.datetime.utcnow()})
                        except:
                            pass
        else:
            if queue.contacts:
                for contact in queue.contacts:
                    try:
                        queue.remove_contact(contact)
                        print({'contact_identification': contact['contact_identification'], 'queue': queue.name, 'event_identification': event.abandon.name, 'event_created_at': datetime.datetime.utcnow()})
                    except:
                        pass

In [8]:
def consumer(agent, queues):
    agents_connected.add_agent(agent.identification)
    while True:
        for queue in queues:
            if queue.contacts:
                contact = queue.get_contact()
                print({'contact_identification': contact['contact_identification'], 'queue': queue.name, 'agent': agent.name, 'event_identification': event.handle.name, 'event_created_at': datetime.datetime.utcnow()})
                time.sleep(contact['handle_time'])
                print({'contact_identification': contact['contact_identification'], 'queue': queue.name, 'agent': agent.name, 'event_identification': event.end.name, 'event_created_at': datetime.datetime.utcnow()})
                break

In [9]:
if __name__ == '__main__':
    agents_connected = agents_connected()
    ################################# first process: queues configurations #################################
    v_sac = queue(1, 'v_sac', channel.inbound.value, 0.8, 20)
    v_suport = queue(1, 'v_suport', channel.inbound.value, 0.7, 30)
    
    threading.Thread(target=system_monitoring, args=(v_sac,)).start()
    threading.Thread(target=system_monitoring, args=(v_suport,)).start()
    ################################# second process: agents configurations #################################
    danilo = agent(1, 'Danilo')
    luana = agent(2, 'Luana')
    daniel = agent(3, 'Daniel')
    angela = agent(3, 'Angela')
    
    threading.Thread(target=consumer, args=(angela, [v_sac, v_suport])).start()
    threading.Thread(target=consumer, args=(danilo, [v_sac, v_suport])).start()
    threading.Thread(target=consumer, args=(luana, [v_sac])).start()
    threading.Thread(target=consumer, args=(daniel, [v_suport])).start()

In [10]:
    ################################# second process: contacts emulation #################################
    for i in range(2):
        v_sac.put_contact(
            {
                "contact_identification": str(uuid.uuid4()),
                "created_at": datetime.datetime.utcnow(),
                "wait_time": round(random.gauss(9, 0), 0),
                "handle_time": round(random.gauss(2, 0), 0)
            }
        )
    for i in range(3):
        v_suport.put_contact(
            {
                "contact_identification": str(uuid.uuid4()),
                "created_at": datetime.datetime.utcnow(),
                "wait_time": round(random.gauss(10, 0), 0),
                "handle_time": round(random.gauss(3, 0), 0)
            }
        )

{'contact_identification': '522189ef-2b95-455c-9359-e7a41e35b47f', 'queue': 'v_sac', 'event_identification': 'start', 'event_created_at': datetime.datetime(2022, 3, 27, 16, 32, 51, 573366)}{'contact_identification': '522189ef-2b95-455c-9359-e7a41e35b47f', 'queue': 'v_sac', 'agent': 'Luana', 'event_identification': 'handle', 'event_created_at': datetime.datetime(2022, 3, 27, 16, 32, 51, 579019)}

{'contact_identification': '5006e492-2526-48b9-a0a1-b190090806dd', 'queue': 'v_sac', 'event_identification': 'start', 'event_created_at': datetime.datetime(2022, 3, 27, 16, 32, 51, 598917)}
{'contact_identification': '804c895b-0869-4070-941e-fb0164c4264f', 'queue': 'v_suport', 'event_identification': 'start', 'event_created_at': datetime.datetime(2022, 3, 27, 16, 32, 51, 599512)}
{'contact_identification': 'b1def052-5ec9-4ed7-a898-90c4d1e48621', 'queue': 'v_suport', 'event_identification': 'start', 'event_created_at': datetime.datetime(2022, 3, 27, 16, 32, 51, 599688)}
{'contact_identification'